In [14]:
import os
import librosa
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [15]:
data_folder = os.path.join(os.path.dirname(os.getcwd()), 'data', 'raw_file')

df = pd.DataFrame({'gwm': [], "mfccs":[], "spectogram":[],"zcr":[],"sc":[],"ss":[],"srf":[],"sf":[],"cv":[],"cd":[] 'emotion': []})

file_num = 1
for filename in os.listdir(data_folder):
    # GWM
    signal, sample_rate = librosa.load(os.path.join(data_folder, filename), sr=None)
    gwm = librosa.feature.rms(y=signal).flatten() 
    
    # EMOCJA
    file_parts = filename.split('-')
    emotion = int(file_parts[2])

    #MFCCS
    mfccs = librosa.feature.mfcc(y=np.squeeze(signal), sr=sample_rate).flatten() 

    #SPECTOGRAM
    spectogram=librosa.feature.melspectrogram(y=signal, sr=sample_rate)
    spectogram_DB = librosa.power_to_db(spectogram, ref=np.max).flatten() 
    
    #Zero Crossing Rate
    zcr = librosa.feature.zero_crossing_rate(signal).flatten()

    #Spectral centroid
    sc = librosa.feature.spectral_centroid(y=signal, sr=sample_rate).flatten()

    #Spectral Spread
    ss = librosa.feature.spectral_bandwidth(y=signal, sr=sample_rate).flatten()

    # Spectral Flux
    sf = librosa.onset.onset_strength(y=signal, sr=sample_rate).flatten()

    # Spectral Roll-Off
    srf = librosa.feature.spectral_rolloff(y=signal, sr=sample_rate).flatten()

    # Chroma Vector
    cv = librosa.feature.chroma_stft(y=signal, sr=sample_rate).flatten()

    #Chroma Deviation
    cv_mean = np.mean(cv, axis=1)  # Średnia dla każdego z 12 chroma
    cd = np.sqrt(np.mean((cv - cv_mean[:, np.newaxis])**2, axis=1))


    # Dodawanie do DataFrame
    temp = pd.DataFrame({'gwm': [gwm],"mfccs": [mfccs],"spectogram":[spectogram_DB],"zcr":[zcr],"sc":[sc],"ss":[ss],"sf":[sf],"srf":[srf],"cv":[cv],"cd":[cd], 'emotion': [emotion]})
    df = df._append(temp, ignore_index=True)
    
    #safety
    file_num += 1
    print(file_num)

SyntaxError: invalid syntax (585337308.py, line 3)

In [ ]:
df.to_csv(r"../data/data_proccesed",index=False)

In [ ]:
df

,GWM,Mfccs,Spectogram,zcr,sc,ss,Emotion
0,"[1.3486991e-06, 1.3486991e-06, 1.3486991e-06, ...","[-861.5326, -861.5326, -861.5326, -861.5326, -...","[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -80...","[0.0, 0.0, 0.0, 0.0, 0.005859375, 0.005859375,...","[12015.384082682827, 12002.424633944627, 11992...","[7001.802469553761, 7005.5081625056355, 7005.8...",1.0
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-820.6302, -820.6302, -820.6302, -820.6302, -...","[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -80...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0
2,"[4.815819e-06, 5.0011104e-06, 5.6821705e-06, 4...","[-787.7085, -787.7085, -787.7085, -787.7085, -...","[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -80...","[0.00634765625, 0.01025390625, 0.01025390625, ...","[11690.832051003013, 11272.503743356196, 11280...","[7156.272130354444, 7059.193491227188, 6967.99...",1.0
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-859.4324, -859.4324, -859.4324, -859.4324, -...","[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -80...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0
4,"[8.818265e-06, 3.8426093e-05, 6.977482e-05, 7....","[-877.39703, -875.6358, -869.74976, -866.593, ...","[-72.53071, -61.344105, -48.231483, -42.507072...","[0.06591796875, 0.07275390625, 0.07275390625, ...","[11773.791581311143, 11208.164633973753, 9450....","[7318.083270032913, 7466.920447772538, 7928.26...",1.0
5,"[0.0, 0.0, 0.0, 9.536743e-07, 5.6420135e-06, 7...","[-937.7508, -937.7508, -937.7508, -937.7508, -...","[-80.0, -80.0, -80.0, -80.0, -80.0, -67.326706...","[0.0, 0.0, 0.0, 0.001953125, 0.001953125, 0.02...","[0.0, 0.0, 0.0, 11924.548962099238, 11781.5506...","[0.0, 0.0, 0.0, 6927.17650348555, 6989.2100743...",1.0
6,"[7.656169e-05, 9.619355e-05, 0.00010942764, 0....","[-795.3941, -770.63165, -763.4191, -763.7233, ...","[-67.59914, -71.40539, -77.71348, -79.87298, -...","[0.14404296875, 0.2158203125, 0.27734375, 0.27...","[8705.836279886142, 8794.441142009411, 8632.20...","[5710.26705379234, 5557.843507527985, 5398.706...",1.0
7,"[4.638329e-05, 6.0462502e-05, 6.440303e-05, 6....","[-846.5356, -865.48584, -878.52423, -878.3453,...","[-47.115437, -50.88907, -45.599205, -45.52865,...","[0.0751953125, 0.0751953125, 0.08447265625, 0....","[6537.15478327908, 8318.067455441977, 8934.863...","[7718.108150538325, 7999.412007810002, 7918.16...",1.0
8,"[0.0005265241, 0.0005914073, 0.0006604255, 0.0...","[-732.3828, -782.88983, -887.6926, -890.5673, ...","[-23.409834, -17.046368, -17.31792, -18.56371,...","[0.00048828125, 0.0029296875, 0.00537109375, 0...","[3503.5840237823663, 2537.5995904860174, 3416....","[5851.316490579468, 5464.433013830445, 6520.47...",1.0
9,"[3.0998082e-05, 4.2382213e-05, 4.4939214e-05, ...","[-745.34045, -742.953, -741.1765, -740.6557, -...","[-70.70176, -62.04086, -59.783047, -62.54836, ...","[0.0673828125, 0.12255859375, 0.1630859375, 0....","[9222.434746603936, 9114.322302215925, 9080.02...","[7608.647987039316, 7757.198160230667, 7763.33...",1.0
